In [1]:

!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets scipy ipywidgets matplotlib
!pip install accelerate sentencepiece optimum auto-gptq trl

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
^C
ERROR: Operation cancelled by user
  distutils: /tmp/pip-build-env-92m1i4oy/normal/lib/python3.10/site-packages
  sysconfig: /tmp/pip-build-env-92m1i4oy/normal/local/lib/python3.10/dist-packages
  distutils: /tmp/pip-build-env-92m1i4oy/normal/lib/python3.10/site-packages
  sysconfig: /tmp/pip-build-env-92m1i4oy/normal/local/lib/python3.10/dist-packages
  user = False
  home = None
  root = None
  prefix = '/tmp/pip-build-env-92m1i4oy/normal'
  distutils: /tmp/pip-build-env-92m1i4oy/overlay/lib/python3.10/site-packages
  sysconfig: /tmp/pip-build-env-92m1i4oy/overlay/local/lib/python3.10/dist-packages
  distutils: /tmp/pip-build-env-92m1i4oy/overlay/lib/python3.10/site-packages
  sysconfig: /tmp/pip-build-env-92m1i4oy/overlay/local/lib/python3.10/dist-packages
  user = False
  home = None
  root = None
  prefix = '/tmp/pip-build-env-92m1i4oy/overlay'


In [1]:

import torch
import os



from peft import (
    LoraConfig,
    #prepare_model_for_kbit_training,
    get_peft_model,
)
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
)
#from trl import SFTTrainer


In [56]:
# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model_name = "models/Mistral-7B-Instruct-v0.2"
#model_name = "models/Mistral-7B-v0.1"
# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    use_cache=False,
    device_map="auto",
)
#model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name,add_bos_token=True, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [88]:
from peft import PeftModel
ft_model = PeftModel.from_pretrained(model,"/netscratch/bverma/results/mistral_instruct_2048_truncate/checkpoint-850")

In [101]:
ft_model= ft_model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:233: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Inference

In [1]:
import os
import json

import torch
import tqdm

import os
split = "dataset_5.json"
data = json.load(open(f"Complete_dataset/{split}"))
files = os.listdir("Complete_dataset/CT json/")
files.remove(".DS_Store")

files_data = {file[:-5]:json.load(open(f"Complete_dataset/CT json/{file}")) for file in files}

data_expanded = []
for _id, value in data.items():
    temp = {}
    temp["id"] = _id
    p_nctid = value["Primary_id"]
    s_nctid = value.get("Secondary_id")
    section_id = value["Section_id"]
    statement = value["Statement"]
    primary_evidence = files_data[p_nctid][section_id]
    temp["statement"] = statement
    temp["primary_evidence"] = primary_evidence
    temp["label"] = value["Label"]
    
    if s_nctid is not None:
        secondary_evidence = files_data[s_nctid][section_id]
        temp["secondary_evidence"] = secondary_evidence
    
    data_expanded.append(temp)

In [2]:
from tqdm import tqdm
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
)
def compute_metrics_decoded(decoded_labs, decoded_preds):

    metrics = {
        "micro_f1": f1_score(decoded_labs, decoded_preds, average="micro"),
        "macro_f1": f1_score(decoded_labs, decoded_preds, average="macro"),
        "precision": precision_score(decoded_labs, decoded_preds, average="micro"),
        "recall": recall_score(decoded_labs, decoded_preds, average="micro"),
        "accuracy": accuracy_score(decoded_labs, decoded_preds),
    }

    return metrics


def get_input_text_p(premise, hypothesis, PROMPT):
    return PROMPT.format(premise=premise, hypothesis=hypothesis)


In [5]:
# BEST

PROMPT = """{premise}
Question: Does this imply that '{hypothesis}' ? Answer Yes or No.
### Answer:
"""
PROMPT_TEST = """
PREMISE: {premise}
HYPOTHESIS: '{hypothesis}'
"""

PROMPT_INST = """<s>[INST] Read the following paragraph and answer the following question.

{premise}
###Question: Does this imply that {hypothesis} ? Answer Yes or No.[/INST]
###Answer:
"""
INFER_PROMPT = """
<s>### Instruction:
Read the input text and answer the following question with Yes or No.

### Input:
{premise}

Question: Does this imply that {hypothesis}?

### Response:
</s>
"""

samples = []
for sample in data_expanded:
    primary_evidence = "".join(sample['primary_evidence'])
    sentence = f"Primary trial evidence are {primary_evidence}"
    secondary_evidence = sample.get("secondary_evidence")
    if secondary_evidence:
        secondary_evidence = "".join(sample['secondary_evidence'])
        sentence = f"{sentence} Secondary trial evidence are {secondary_evidence}"
    input_text = get_input_text_p(sentence, sample['statement'], PROMPT_TEST)
    if 'label' in sample:
        temp = {"text":input_text, "label":sample['label'], "id": sample['id']}
    else:
        temp = {"text":input_text, "id": sample['id']}
    samples.append(temp)





[{'id': '9159',
  'statement': 'the primary trial participants are given saracatinib PO for a maximum of 28 weeks.',
  'primary_evidence': ['INTERVENTION 1: ',
   '  Treatment (Enzyme Inhibitor Therapy)',
   '  Patients receive saracatinib PO on days 1-28. Treatment repeats every 28 days in the absence of disease progression or unacceptable toxicity.'],
  'label': 'Entailment'},
 {'id': '9162',
  'statement': 'Stephanie has been living with her boyfriend for 2 years, she is eligible for the primary trial.',
  'primary_evidence': ['Inclusion Criteria:',
   '  Female',
   '  Age > 21 years',
   '  Has diagnosis of non-recurrent stage I-III breast cancer',
   '  Completed active treatment (e.g., chemotherapy, radiation therapy, surgery) 6 months-5 years ago (current use of endocrine therapy is acceptable)',
   '  Has a partner or spouse who is > 21',
   '  Lives with a romantic partner > 6 months',
   '  Score of > 3 on Patient Care Monitor Sexual Concerns screening item',
   '  No hearin

In [66]:
res = []
labels = []
for sample in tqdm(samples):
    labels.append(sample['label'])
    inputs = tokenizer(sample['text'], return_tensors="pt")
    with torch.inference_mode():
        outputs = ft_model.generate(
        input_ids=inputs["input_ids"].to("cuda"), 
        max_new_tokens=len(inputs["input_ids"])+8,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        top_k=40,
        repetition_penalty=1.1,
        )
        
    res.append(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])



  0%|          | 0/200 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 1/200 [00:01<03:55,  1.18s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  1%|          | 2/200 [00:03<06:06,  1.85s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  1%|          | 2/200 [00:04<06:45,  2.05s/it]


KeyboardInterrupt: 

In [114]:
pred = []
labs = []
for i, r in enumerate(res):
    r = r.split('Answer:')[1]
    if 'yes' in r.lower():
        pred.append('Entailment')
        labs.append(labels[i])
    elif 'no' in r.lower():
        pred.append('Contradiction')
        labs.append(labels[i])

result = compute_metrics_decoded(labs, pred)
print(result)

{'micro_f1': 0.7299999999999999, 'macro_f1': 0.7295673076923077, 'precision': 0.73, 'recall': 0.73, 'accuracy': 0.73}


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(labs, pred))

               precision    recall  f1-score   support

Contradiction       0.71      0.69      0.70        99
   Entailment       0.70      0.72      0.71        99

     accuracy                           0.70       198
    macro avg       0.70      0.70      0.70       198
 weighted avg       0.70      0.70      0.70       198



In [110]:
sample = samples[0]['text']

inputs = tokenizer(sample, return_tensors="pt")
ft_model.eval()
with torch.no_grad():
    outputs = ft_model.generate(
        input_ids=inputs["input_ids"].to("cuda"), 
        max_new_tokens=1,#len(inputs["input_ids"])+8,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        top_k=40,
        repetition_penalty=1.1,
        output_scores=True, return_dict_in_generate=True
        )
    #print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [111]:
import numpy as np
transition_scores = model.compute_transition_scores(
    outputs.sequences, outputs.scores, normalize_logits=True
)
# input_length is the length of the input prompt for decoder-only models, like the GPT family, and 1 for
# encoder-decoder models, like BART or T5.
input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
generated_tokens = outputs.sequences[:, input_length:]
for tok, score in zip(generated_tokens[0], transition_scores[0]):
    # | token | token string | logits | probability
    print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.cpu().numpy():.3f} | {np.exp(score.cpu().numpy()):.2%}")



|  5613 | Yes      | -0.289 | 74.90%


In [94]:
"score": float(np.exp(transition_scores.cpu())), "output": tokenizer.decode(generated_tokens[0])

0.5348114371299744

In [99]:
tokenizer.decode(generated_tokens[0])

'Yes'

In [8]:
import torch
from torch.nn.functional import softmax

softmax(outputs['scores'][0][torch.isfinite(outputs['scores'][0])], dim=-1)

NameError: name 'outputs' is not defined

In [42]:
results = dd(lambda: dd(lambda: dd(int)))

In [15]:
## INFERENCE
from collections import defaultdict as dd
from tqdm import tqdm
res = dd()
for sample in tqdm(samples):
    id_ = sample['id']
    inputs = tokenizer(sample['text'], return_tensors="pt")
    with torch.inference_mode():
        outputs = ft_model.generate(
        input_ids=inputs["input_ids"].to("cuda"), 
        max_new_tokens=len(inputs["input_ids"])+8,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        top_k=40,
        repetition_penalty=1.1,
        )
    res[id_] = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]



pred = []

for id_, r in res.items():
    r = r.split('Response:')[-1]
    if 'yes' in r.lower():
        results[id_]['prediction']['Entailment'] += 1 
        #pred.append('Entailment')
        #labs.append(labels[i])
    elif 'no' in r.lower():
        results[id_]['prediction']['Contradiction'] += 1 

  0%|          | 0/2142 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 1/2142 [00:02<1:31:09,  2.55s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 2/2142 [00:04<1:16:13,  2.14s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 3/2142 [00:06<1:17:18,  2.17s/it]The attention mask and the pad token id were not set. As a consequenc

In [16]:
json.dump(results, open('results.json', 'w'), indent=4)

In [112]:
from collections import defaultdict as dd
results = dd(lambda: dd(lambda: dd(lambda: {'count': 0, 'score': []})))

In [113]:
from collections import defaultdict as dd
from tqdm import tqdm
import numpy as np

k=0
while (k < 5):
    res = dd()
    labels = []
    for sample in tqdm(samples):
        id_ = sample['id']
        labels.append(sample['label'])
        inputs = tokenizer(sample['text'], return_tensors="pt")
        with torch.inference_mode():
            outputs = ft_model.generate(
            input_ids=inputs["input_ids"].to("cuda"), 
            max_new_tokens=1,#len(inputs["input_ids"])+8,
            do_sample=True,
            temperature=0.7,
            top_p=0.95,
            top_k=40,
            repetition_penalty=1.1,
            output_scores=True, return_dict_in_generate=True,
            )
            transition_scores = model.compute_transition_scores(
            outputs.sequences, outputs.scores, normalize_logits=True
            )
            # input_length is the length of the input prompt for decoder-only models, like the GPT family, and 1 for
            # encoder-decoder models, like BART or T5.
            input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
            generated_tokens = outputs.sequences[:, input_length:]
            res[id_] = {"score": float(np.exp(transition_scores.cpu())), "output": tokenizer.decode(generated_tokens[0])} #tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]



    pred = []

    for id_, r in res.items():
        word = r['output']
        if 'yes' in word.lower():
            results[id_]['prediction']['Entailment']['count'] += 1 
            results[id_]['prediction']['Entailment']['score'].append(r['score']) 
            #pred.append('Entailment')
            #labs.append(labels[i])
        elif 'no' in word.lower():
            results[id_]['prediction']['Contradiction']['count'] += 1
            results[id_]['prediction']['Contradiction']['score'].append(r['score']) 
    
    k += 1

  0%|          | 0/200 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 1/200 [00:00<00:27,  7.20it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  1%|          | 2/200 [00:00<00:39,  4.98it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  2%|▏         | 3/200 [00:00<00:38,  5.08it/s]The attention mask and the pad token id were not set. As a consequence, you may

In [114]:
labels = dd(str)
for sample in tqdm(samples):
    id_ = sample['id']
    labels[id_] = sample['label']

100%|██████████| 200/200 [00:00<00:00, 515587.46it/s]


In [14]:
results

defaultdict(<function __main__.<lambda>()>,
            {'1adc970c-d433-44d0-aa09-d3834986f7a2': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'prediction': defaultdict(<function __main__.<lambda>.<locals>.<lambda>.<locals>.<lambda>()>,
                                      {'Entailment': {'count': 3,
                                        'score': [0.7767490744590759,
                                         0.7767490744590759,
                                         0.7767490744590759]},
                                       'Contradiction': {'count': 2,
                                        'score': [0.14562122523784637,
                                         0.14562122523784637]}})}),
             '6b9162d0-0816-46d4-81af-c60028dcc63b': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'prediction': defaultdict(<function __main__.<lambda>.<locals>.<lambda>.<locals>.<lambda>()>,
                  

In [115]:
def get_predicted_label(instance_data):
    # Get the 'prediction' sub-dictionary
    prediction_data = instance_data.get('prediction', {})

    # Initialize variables to store the predicted label and its count
    predicted_label = None
    max_count = 0

    # Iterate through the prediction data to find the label with the highest count
    for label, label_data in prediction_data.items():
        count = label_data.get('count', 0)
        if count > max_count:
            max_count = count
            predicted_label = label

    return predicted_label

# Create a list to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Iterate through your data
for instance_id, instance_data in results.items():
    gold_label = labels[instance_id]  # You need to implement a function to get the gold label based on instance_id
    true_labels.append(gold_label)

    predicted_label = get_predicted_label(instance_data)  # You need to implement a function to get the predicted label based on instance_data
    predicted_labels.append(predicted_label)

from sklearn.metrics import classification_report

print(classification_report(true_labels, predicted_labels))

               precision    recall  f1-score   support

Contradiction       0.66      0.77      0.71       100
   Entailment       0.73      0.61      0.66       100

     accuracy                           0.69       200
    macro avg       0.69      0.69      0.69       200
 weighted avg       0.69      0.69      0.69       200



In [17]:
incorrect_predictions = []

# Iterate through your data
for instance_id, instance_data in results.items():
    gold_label = labels[instance_id]  # You need to implement a function to get the gold label based on instance_id

    # Get the predicted label
    predicted_label = get_predicted_label(instance_data)  # You need to implement a function to get the predicted label based on instance_data

    # Check if the predicted label is different from the gold label
    if predicted_label != gold_label:
        incorrect_predictions.append(instance_id)




In [22]:
incorrect_predictions

['1adc970c-d433-44d0-aa09-d3834986f7a2',
 '3facad41-0221-42f8-834d-470e65c4aad5',
 '24b85b44-b9e6-4c28-b3aa-1bd97102b7f1',
 '0ddb07ae-6e70-436d-8723-f609e59c57da',
 '2655c892-216e-4b96-a8f2-6f64403215c5',
 'f46a7d27-a94f-4389-a1a0-4a8f5177b354',
 '4f705481-5dd0-414c-b375-c56e8c235b42',
 '47ba01d6-edc7-4acb-ab49-b0e86f4eeaf7',
 '8c6314f7-34f5-42a7-ba66-b95d050a6315',
 '734423d5-4b47-4448-ab98-811015d626f0',
 '89f610a7-9e81-43a4-8c1e-5503233b2be1',
 '8ddf8b88-6e06-4c71-9e28-45f73a637af0',
 '9abb10d7-496e-44fd-915d-d86aedf6c9b6',
 '17bbea92-8f92-41f3-9b9d-cf6e62cf6798',
 '4dfff980-6a2b-4f01-9194-6ec831d6fed7',
 '8d9f94e3-871b-4fb4-a522-0a73d9d0ec3c',
 '251a20d8-2555-4929-9d01-c02311bdc93f',
 '080da381-b5dd-46d3-99c4-18112da153c1',
 'd1b7105d-0b7d-4b10-92a7-d63243ff0629',
 'b1dd7632-ed92-40a9-9293-bce68619c658',
 'b9e29ea8-6847-40d4-b5c3-81c496101695',
 '4a41ac73-538a-43e5-b7ce-8c8e2d56286e',
 '48f516ff-abb4-4312-9b00-02f53fb16218',
 '63a0a9fc-80b9-450f-b2c6-7aee37ce26a9',
 '75cf4ec2-9f2b-

In [25]:
results['24b85b44-b9e6-4c28-b3aa-1bd97102b7f1']

defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
            {'prediction': defaultdict(<function __main__.<lambda>.<locals>.<lambda>.<locals>.<lambda>()>,
                         {'Contradiction': {'count': 4,
                           'score': [0.873450517654419,
                            0.873450517654419,
                            0.873450517654419,
                            0.873450517654419]}})})

In [28]:
import pandas as pd
df = pd.DataFrame(data_expanded)

In [31]:
df

,id,statement,primary_evidence,label,secondary_evidence
0,1adc970c-d433-44d0-aa09-d3834986f7a2,there is a 13.2% difference between the result...,"[Outcome Measurement: , Event-free Survival,...",Contradiction,NaN
1,6b9162d0-0816-46d4-81af-c60028dcc63b,Patients with significantly elevated ejection ...,"[Inclusion criteria:, Inclusion Criteria:, F...",Contradiction,[Premenopausal women 55 years of age or younge...
2,0b6cc8e3-69ee-4a91-b93d-2ad3fddce65f,a significant number of the participants in th...,"[Adverse Events 1:, Total: 20/167 (11.98%), ...",Contradiction,"[Adverse Events 1:, Total: 112/458 (24.45%),..."
3,cc1f712a-2116-4e40-9810-f315e3fa5ff8,the primary trial does not report the PFS or o...,"[Outcome Measurement: , Local Control Using ...",Entailment,NaN
4,904061c0-14fa-4f13-9118-9a41e24fa8eb,Prior treatment with fulvestrant or with a pho...,"[Inclusion Criteria:, Postmenopausal women w...",Contradiction,NaN
...,...,...,...,...,...
195,d310ec4e-993e-4827-8dc5-9aca053972db,The the primary trial intervention involves on...,"[INTERVENTION 1: , Letrozole, Participants...",Contradiction,"[INTERVENTION 1: , Sentinel Lymph Node Biops..."
196,a5617ae4-05a3-42d0-9e14-141de5f8c010,the secondary trial reported 1 single case of ...,"[Adverse Events 1:, Total: 16/48 (33.33%), ...",Entailment,"[Adverse Events 1:, Total: 21/519 (4.05%), ..."
197,42d1fcd3-8faa-4065-bbba-42cc90ab67fb,the secondary trial and the primary trial do n...,"[Outcome Measurement: , Number of Patients W...",Entailment,"[Outcome Measurement: , Disease Free Surviva..."
198,d01fda83-5dc8-4ad5-92b8-7553dabd7046,the outcome measurement of the primary trial i...,"[Outcome Measurement: , Progression-free Sur...",Entailment,NaN


In [40]:
df[df['id'] == '24b85b44-b9e6-4c28-b3aa-1bd97102b7f1']['primary_evidence'].values

array([list(['Inclusion Criteria Phase 1', '  Age greater than/equal to 18 years', '  Histologic diagnosis of palpable invasive breast cancer or ductal carcinoma in situ', '  Patient desire to undergo breast surgery', '  3. Patients will have provided informed consent to participate, documented by their signature on the study consent form 4. The cancer enhances on breast MRI imaging.', '  Inclusion Criteria Phase 2', '  Age greater than/equal to 18 years', '  Histologic diagnosis of invasive breast cancer or ductal carcinoma in situ', '  The tumor is visible and enhances on prone MRI and is >1 cm in greatest diameter.', '  . Determination by the surgeon that the neoplasm is non-palpable.A patient with a palpable hematoma from core biopsy, but a non-palpable neoplasm, will be eligible for study', '  Patient desire to undergo breast conserving surgery', '  Patients will have provided informed consent to participate, documented by their signature on the study consent form.The process of i

In [9]:
from sklearn.metrics import classification_report

print(classification_report(labs, pred))

               precision    recall  f1-score   support

Contradiction       0.71      0.73      0.72       100
   Entailment       0.72      0.70      0.71       100

     accuracy                           0.71       200
    macro avg       0.72      0.71      0.71       200
 weighted avg       0.72      0.71      0.71       200



In [26]:
from sklearn.metrics import classification_report

print(classification_report(labs, pred))

               precision    recall  f1-score   support

Contradiction       0.00      0.00      0.00       100
   Entailment       0.50      1.00      0.67       100

     accuracy                           0.50       200
    macro avg       0.25      0.50      0.33       200
 weighted avg       0.25      0.50      0.33       200



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
from sklearn.metrics import classification_report

print(classification_report(labs, pred))

               precision    recall  f1-score   support

Contradiction       0.81      0.55      0.65       100
   Entailment       0.66      0.87      0.75       100

     accuracy                           0.71       200
    macro avg       0.73      0.71      0.70       200
 weighted avg       0.73      0.71      0.70       200



In [23]:
from sklearn.metrics import classification_report

print(classification_report(labs, pred))

               precision    recall  f1-score   support

Contradiction       0.72      0.74      0.73       100
   Entailment       0.73      0.71      0.72       100

     accuracy                           0.73       200
    macro avg       0.73      0.72      0.72       200
 weighted avg       0.73      0.72      0.72       200



In [61]:
from sklearn.metrics import classification_report

print(classification_report(labs, pred))

               precision    recall  f1-score   support

Contradiction       0.59      0.89      0.71       100
   Entailment       0.78      0.38      0.51       100

     accuracy                           0.64       200
    macro avg       0.68      0.64      0.61       200
 weighted avg       0.68      0.64      0.61       200



Llama2 13B

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline
tokenizer = AutoTokenizer.from_pretrained("TheBloke/Llama-2-13B-chat-GPTQ")

model = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-13B-chat-GPTQ", device_map='auto')

pipe = pipeline(
    task='text-generation',
    model=model,
    tokenizer=tokenizer,
    max_length=2048,
    temperature=0.1,
    top_p=0.95,
    repetition_penalty=1.15
)

In [8]:
# BEST

PROMPT = """{premise}'
Question: Does this imply that '{hypothesis}' ? Answer Yes or No.
### Answer:
"""

samples = []
for sample in data_expanded:
    primary_evidence = "".join(sample['primary_evidence'])
    sentence = f"Primary trial evidence are {primary_evidence}"
    secondary_evidence = sample.get("secondary_evidence")
    if secondary_evidence:
        secondary_evidence = "".join(sample['secondary_evidence'])
        sentence = f"{sentence} Secondary trial evidence are {secondary_evidence}"
    input_text = get_input_text_p(sentence, sample['statement'], PROMPT)
    temp = {"text":input_text, "label":sample['label']}
    samples.append(temp)



res = []
labels = []
for sample in tqdm(samples[:200]):
    labels.append(sample['label'])
    with torch.inference_mode():
        output = pipe(sample['text'])
        
    res.append(output[0]['generated_text'])

pred = []
for r in res:
    r = r.split('### Answer:')[1]
    if 'Yes' in r:
        pred.append('Entailment')
    elif 'No' in r:
        pred.append('Contradiction')

result = compute_metrics_decoded(labels, pred)
print(result)

  0%|          | 0/200 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-b

{'micro_f1': 0.54, 'macro_f1': 0.5066495066495067, 'precision': 0.54, 'recall': 0.54, 'accuracy': 0.54}


In [9]:
result

{'micro_f1': 0.54,
 'macro_f1': 0.5066495066495067,
 'precision': 0.54,
 'recall': 0.54,
 'accuracy': 0.54}

Mistaral-7B

In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline
tokenizer = AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-OpenOrca-GPTQ")

model = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-OpenOrca-GPTQ", device_map='auto')
from auto_gptq import exllama_set_max_input_length
model = exllama_set_max_input_length(model, max_input_length=8048)
pipe = pipeline(
    task='text-generation',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=64,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


In [16]:
# BEST

PROMPT = """{premise}'
Question: Does this imply that '{hypothesis}' ? Answer Yes or No.
### Answer:
"""

samples = []
for sample in data_expanded:
    primary_evidence = "".join(sample['primary_evidence'])
    sentence = f"Primary trial evidence are {primary_evidence}"
    secondary_evidence = sample.get("secondary_evidence")
    if secondary_evidence:
        secondary_evidence = "".join(sample['secondary_evidence'])
        sentence = f"{sentence} Secondary trial evidence are {secondary_evidence}"
    input_text = get_input_text_p(sentence, sample['statement'], PROMPT)
    temp = {"text":input_text, "label":sample['label']}
    samples.append(temp)



res = []
labels = []
for sample in tqdm(samples):
    labels.append(sample['label'])
    with torch.inference_mode():
        output = pipe(sample['text'])
        
    res.append(output[0]['generated_text'])

pred = []
for r in res:
    r = r.split('### Answer:')[1]
    if 'Yes' in r:
        pred.append('Entailment')
    elif 'No' in r:
        pred.append('Contradiction')

result = compute_metrics_decoded(labels, pred)
print(result)

  0%|          | 0/1700 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
  1%|          | 10/1700 [00:02<05:05,  5.53it/s]/opt/conda/lib/python3.8/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 1700/1700 [06:16<00:00,  4.51it/s]

{'micro_f1': 0.5835294117647059, 'macro_f1': 0.5588616901652106, 'precision': 0.5835294117647059, 'recall': 0.5835294117647059, 'accuracy': 0.5835294117647059}


: 

In [13]:
result

{'micro_f1': 0.65,
 'macro_f1': 0.6224786970121885,
 'precision': 0.65,
 'recall': 0.65,
 'accuracy': 0.65}

Flan-T5-XXL

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xxl")

model = AutoModelForCausalLM.from_pretrained("google/flan-t5-xxl", device_map='auto')

pipe = pipeline(
    task='text-generation',
    model=model,
    tokenizer=tokenizer,
    max_length=2048,
    temperature=0.1,
    top_p=0.95,
    repetition_penalty=1.15
)

In [ ]:
# BEST

PROMPT = """{premise}'
Question: Does this imply that '{hypothesis}' ? Answer Yes or No.
### Answer:
"""

samples = []
for sample in data_expanded:
    primary_evidence = "".join(sample['primary_evidence'])
    sentence = f"Primary trial evidence are {primary_evidence}"
    secondary_evidence = sample.get("secondary_evidence")
    if secondary_evidence:
        secondary_evidence = "".join(sample['secondary_evidence'])
        sentence = f"{sentence} Secondary trial evidence are {secondary_evidence}"
    input_text = get_input_text_p(sentence, sample['statement'], PROMPT)
    temp = {"text":input_text, "label":sample['label']}
    samples.append(temp)



res = []
labels = []
for sample in tqdm(samples[:200]):
    labels.append(sample['label'])
    with torch.inference_mode():
        output = pipe(sample['text'])
        
    res.append(output[0]['generated_text'])

pred = []
for r in res:
    r = r.split('### Answer:')[1]
    if 'Yes' in r:
        pred.append('Entailment')
    elif 'No' in r:
        pred.append('Contradiction')

result = compute_metrics_decoded(labels, pred)
print(result)